# Import data

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import os
from tqdm import tqdm

In [2]:
df=pd.read_csv(path+"20170101.csv")
df=df.drop('Unnamed: 0',1)

FileNotFoundError: [Errno 2] File b'drive/My Drive/1001_project_Total/Data/20170101.csv' does not exist: b'drive/My Drive/1001_project_Total/Data/20170101.csv'

In [7]:
df.columns

Index(['profile', 'category1', 'canonicalUrl', 'referrerHostClass',
       'sessionStop', 'userId', 'publishtime', 'sessionStart', 'referrerUrl',
       'keywords', 'id', 'eventId', 'city', 'title', 'url', 'country',
       'region', 'author', 'referrerSearchEngine', 'deviceType', 'time', 'os',
       'activeTime'],
      dtype='object')

# Adding Sessions

In [8]:
t=df.sort_values(["userId","time"])

# according from the dataset description: There is the threshold when computing the “activeTime”, when it is too short then it won’t be account in original dataset
t["activeTime"]=t["activeTime"].fillna(0)

# find the sessions with total time more than 0 and less than 45 minutes
t["sessionNo"]=t["sessionStart"].cumsum()
t["activeTotalTime"]=t.groupby('sessionNo')['activeTime'].transform('sum')
print("The number of total events by users is: ",len(t))
print("The number of sessions is: ",max(t["sessionNo"]))
print("The number of unique users is: ",len(t["userId"].unique()))
t["sessionNo"]=t["sessionStart"].cumsum()
t=t[t["activeTotalTime"]>0]
t=t[t["activeTotalTime"]<=2700]

The number of total events by users is:  1513739
The number of sessions is:  518675
The number of unique users is:  248892


In [9]:
print("The number of total events by users is: ",len(t))
print("The number of sessions is: ",len(t["sessionNo"].unique()))
print("The number of unique users is: ",len(t["userId"].unique()))

The number of total events by users is:  1245204
The number of sessions is:  293529
The number of unique users is:  148781


In [10]:
t[["sessionNo","id","time","sessionStart","sessionStop","activeTime","activeTotalTime"]].iloc[0:10]

,sessionNo,id,time,sessionStart,sessionStop,activeTime,activeTotalTime
1348419,2,NaN,1483305709,True,False,28.0,81.0
1349279,2,b28e7c163c39941aa1cbd0b7b3a821576771f893,1483305738,False,False,35.0,81.0
1350387,2,NaN,1483305778,False,False,6.0,81.0
1350581,2,4c1af3633d74ba9eb77b2c07c7a8171ba27b10f7,1483305786,False,False,9.0,81.0
1350878,2,NaN,1483305798,False,False,3.0,81.0
1350954,2,9d615dd08d92c8e9670fb72b5c78cbc6b52501c4,1483305801,False,False,0.0,81.0
1351021,2,NaN,1483305804,False,True,0.0,81.0
326027,4,NaN,1483260167,True,False,86.0,86.0
328012,4,68d1503c73ad169dcfff48214fd0274c4d612e63,1483260265,False,True,0.0,86.0
609122,5,NaN,1483271301,True,False,66.0,145.0


In [11]:
t.dropna(subset=["id"],inplace=True)
articles=t["id"].unique()
d={}
for i in range(len(articles)):
    article=articles[i]
    d[article]=i
t["articleId"]=t["id"].map(d)
print("The length of t is: ",len(t))
t.head()

The length of t is:  374487


,profile,category1,canonicalUrl,referrerHostClass,sessionStop,userId,publishtime,sessionStart,referrerUrl,keywords,id,eventId,city,title,url,country,region,author,referrerSearchEngine,deviceType,time,os,activeTime,sessionNo,activeTotalTime,articleId
1349279,"[{'item': '0', 'groups': [{'count': 1, 'group'...",100sport|vintersport,http://www.adressa.no/100Sport/vintersport/lan...,search,False,cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov,2017-01-01T14:33:41.000Z,False,http://adressa.no,"Petter Northug,Thomas Northug",b28e7c163c39941aa1cbd0b7b3a821576771f893,950557625,oslo,To av tre Northug-brødre tilbake for fullt: Ti...,http://adressa.no/100sport/langrenn_old/to-av-...,no,oslo,ole k. sagbakken,Internal,Desktop,1483305738,Windows,35.0,2,81.0,0
1350581,"[{'item': '0', 'groups': [{'count': 1, 'group'...",NaN,http://www.adressa.no/nyheter/moreromsdal/2017...,search,False,cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov,NaN,False,http://adressa.no,NaN,4c1af3633d74ba9eb77b2c07c7a8171ba27b10f7,806540643,oslo,Kvinne omkommet etter ulykke på E39,http://adressa.no/nyheter/moreromsdal/2017/01/...,no,oslo,NaN,Internal,Desktop,1483305786,Windows,9.0,2,81.0,1
1350954,"[{'item': '0', 'groups': [{'count': 1, 'group'...",NaN,http://www.adressa.no/nyheter/nordtrondelag/20...,internal,False,cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov,NaN,False,http://adressa.no,NaN,9d615dd08d92c8e9670fb72b5c78cbc6b52501c4,1926247176,oslo,Bil av veien i Meråker,http://adressa.no/nyheter/nordtrondelag/2017/0...,no,oslo,NaN,NaN,Desktop,1483305801,Windows,0.0,2,81.0,2
328012,"[{'item': '0', 'groups': [{'count': 1, 'group'...",100sport|vintersport,http://www.adressa.no/100Sport/vintersport/lan...,search,True,cx:02b046b0db0d02e5dadcdb2d4c3d7e53:24ik5bzhn1crw,2016-12-31T18:06:21.000Z,False,http://adressa.no,"Tour de ski,Langrenn",68d1503c73ad169dcfff48214fd0274c4d612e63,1292992944,brumunddal,Her koker det over for Tønseth. Så stakk han f...,http://adressa.no/100sport/vintersport/langren...,no,hedmark,birger løfaldli,Internal,Desktop,1483260265,Windows,0.0,4,86.0,3
613313,"[{'item': '0', 'groups': [{'count': 1, 'group'...",nyheter|trondheim,http://www.adressa.no/nyheter/trondheim/2016/1...,internal,True,cx:02b046b0db0d02e5dadcdb2d4c3d7e53:24ik5bzhn1crw,2016-11-01T09:18:19.000Z,False,http://adressa.no/nyheter/trondheim/2016/12/31...,"utenriks,innenriks,trondheim,E6,midtbyen,bybra...",7fee41607635cb686671c95fa0dabe16a2a128f5,2095395787,brumunddal,Nå må du punge ut for piggdekkene,http://adressa.no/nyheter/trondheim/2016/11/01...,no,hedmark,martin andersen,NaN,Desktop,1483271454,Windows,0.0,5,145.0,4


In [ ]:
t.to_csv(path+"20170101_prestage.csv")

In [13]:
len(t)

374487